## Build an MTH5 and Operate the Aurora Pipeline

This notebook pulls MT miniSEED data from the IRIS Dataselect web service and produces MTH5 out of it. It outlines the process of making an MTH5 file, generating a processing config, and running the Aurora processor.

It assumes that aurora, mth5, and mt_metadata have all been installed.

In [ ]:
# Required imports for theh program. 
from pathlib import Path
import sys
import pandas as pd
import mth5
from mth5.clients.make_mth5 import MakeMTH5
from mth5 import mth5, timeseries
from mt_metadata.utils.mttime import get_now_utc, MTime
from aurora.config.config_creator import ConfigCreator
from aurora.pipelines.process_mth5 import process_mth5_run

### Set up MTH5 object and retrieve data
To build an MTH5 file with data extracted from IRIS, first initialize an MTH5 object that will be used to grab the requested data from the specified client. We create a dataframe of the targets and time frames of the desired data, which is then passed along to the MTH5 object.

In [ ]:
# Set path so MTH5 file builds to current working directory. 
default_path = Path().cwd()

In [ ]:
# Initialize the Make MTH5 code. 
m = MakeMTH5(mth5_version='0.1.0')
m.client = "IRIS"

In [ ]:
# Generate data frame of FDSN Network, Station, Location, Channel, Startime, Endtime codes of interest

ZUCAS04LQ1 = ['8P', 'CAS04', '', 'LQE', '2020-06-02T19:00:00', '2020-07-13T19:00:00']
ZUCAS04LQ2 = ['8P', 'CAS04', '', 'LQN', '2020-06-02T19:00:00', '2020-07-13T19:00:00']
ZUCAS04BF1 = ['8P', 'CAS04', '', 'LFE', '2020-06-02T19:00:00', '2020-07-13T19:00:00']
ZUCAS04BF2 = ['8P', 'CAS04', '', 'LFN', '2020-06-02T19:00:00', '2020-07-13T19:00:00']
ZUCAS04BF3 = ['8P', 'CAS04', '', 'LFZ', '2020-06-02T19:00:00', '2020-07-13T19:00:00']

request_list = [ZUCAS04LQ1, ZUCAS04LQ2, ZUCAS04BF1, ZUCAS04BF2, ZUCAS04BF3]

# Turn list into dataframe
request_df =  pd.DataFrame(request_list, columns=m.column_names)

In [ ]:
# Inspect the dataframe
print(request_df)

You can view the inventory of the targets and timespans that the client (IRIS, in this case) has available. data=False will display just the metadata and is quicker, data=True will also request information about the data extents that are available.  

In [ ]:
# Request the inventory information from IRIS
inventory = m.get_inventory_from_df(request_df, data=False)

In [ ]:
# Inspect the inventory
inventory

With the mth5 object set, we are ready to actually request the data from the fdsn client (IRIS) and save it to an MTH5 file.  This process builds an MTH5 file and can take some time depending on how much data is requested.  

Note: interact keeps the MTH5 open after it is done building


In [ ]:
mth5_object = m.make_mth5_from_fdsnclient(request_df, interact=True)

In [ ]:
# mth5_object.open_mth5(h5_path, 'w')
# h5_path = str(default_path)+'/8P_CAS04.h5'
#mth5_object.close_mth5()

### Examine and Update the MTH5 object

With the open MTH5 Object, we can start to examine what is in it. For example, we retrieve the filename and file_version. You can additionally do things such as getting the station information and edit it by setting a new value, in this case the declination model. 

In [ ]:
mth5_object

In [ ]:
# Collect information from the MTh5 Object and use it in the config files. 
mth5_filename = mth5_object.filename
version = mth5_object.file_version
print(mth5_filename)

In [ ]:
# Edit and update the MTH5 metadata 
s = mth5_object.get_station("CAS04")
print(s.metadata.location.declination.model)
s.metadata.location.declination.model = 'IGRF'
print(s.metadata.location.declination.model)
s.write_metadata()    # writes to file mth5_filename

Now we look more closely at the metadata. Start by calling .summarize() to initialize the channel_summary if we have not yet closed the mth5 object (we have not).  Then by calling .to_dataframe() we can start calling on attributes such as run or sample_rate. 

In [ ]:
# Get the available stations and runs from the MTH5 object
mth5_object.channel_summary.summarize()
ch_summary = mth5_object.channel_summary.to_dataframe()

In [ ]:
ch_summary

In [ ]:
available_runs = ch_summary.run.unique()
sr = ch_summary.sample_rate.unique()
if len(sr) != 1:
    print('Only one sample rate per run is available')
available_stations = ch_summary.station.unique()

In [ ]:
sr[0]

In [ ]:
available_stations[0]

In [ ]:
mth5_object

### Generate an Aurora Configuration file using MTH5 as an input

Up to this point, we have used mth5 and mt_metadata, but haven't yet used aurora. So we will use the MTH5 that we just created (and examined and updated) as input into Aurora.

First, we get some fields from the MTH5 and input those into an aurora Configuration file and then save that json file.

In [ ]:
station_id = available_stations[0]
run_id = available_runs[0]
sample_rate = sr[0]
config_maker = ConfigCreator()
config_path = config_maker.create_run_config(station_id, run_id, mth5_filename, sample_rate)


In [ ]:
config_path

Run the Aurora Pipeline using the input MTH5 and Confiugration File, outputting transfer functions.

In [ ]:
show_plot='True'   # change to 'False' to omit plots

In [ ]:
tf_cls = process_mth5_run(
        config_path,
        run_id,
        mth5_path=mth5_filename,
        units="MT",
        show_plot=show_plot,
        z_file_path=None,
        return_collection=False,
    )

In [ ]:
type(tf_cls)

Write the transfer functions generated by the Aurora pipeline, showing examples of various output formats.

In [ ]:
tf_cls.write_tf_file(fn="emtfxml_test.xml", file_type="emtfxml")

In [ ]:
tf_cls.write_tf_file(fn="edi_test.edi", file_type="edi")

In [ ]:
tf_cls.write_tf_file(fn="zmm_test.zmm", file_type="zmm")